In [ ]:
"""
Purpose: To put all of the rules together
to autoatically proofread a cell


"""

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from os import sys
sys.path.append("/meshAfterParty/")

In [ ]:
from importlib import reload

import neuron_utils as nru

import neuron

import neuron_visualizations as nviz

import time

import datajoint_utils as du

import numpy as np

import proofreading_utils as pru

import preprocessing_vp2 as pre

# -- For the axon classification --

import neuron_searching as ns

import skeleton_utils as sk

import numpy_utils as nu

import networkx_utils as xu

import system_utils as su

import classification_utils as clu
import proofreading_utils as pru

In [ ]:
minnie,schema = du.configure_minnie_vm()

# Importing the Neuron

In [ ]:
curr_table = minnie.Decomposition & "process_version = 4" & "n_limbs>5" 
curr_table

In [ ]:
segment_id = 864691135403709422

In [ ]:
neuron_obj_original = (curr_table & dict(segment_id=segment_id)).fetch1("decomposition")

In [ ]:
nviz.visualize_neuron_lite(neuron_obj_original)

# Splitting the Neuron

In [ ]:
split_results = pru.multi_soma_split_suggestions(neuron_obj_original,
                                                )

In [ ]:
nviz.plot_split_suggestions_per_limb(neuron_obj_original,
                                    split_results)

In [ ]:
(neuron_list,
neuron_list_errored_limbs_area,
 neuron_list_errored_limbs_skeletal_length,
neuron_list_n_multi_soma_errors,
neuron_list_n_same_soma_errors) = pru.split_neuron(neuron_obj_original,
                limb_results=split_results,
                               verbose=False,
                                return_error_info=True
                                    )

print(f" # of neurons = {len(neuron_list)}")
print(f"neuron_list_errored_limbs_area = {neuron_list_errored_limbs_area}")
print(f"neuron_list_errored_limbs_skeletal_length = {neuron_list_errored_limbs_skeletal_length}")
print(f"neuron_list_n_multi_soma_errors = {neuron_list_n_multi_soma_errors}")
print(f"neuron_list_n_same_soma_errors = {neuron_list_n_same_soma_errors}")

In [ ]:
neuron_obj_1 = neuron_list[0]
nviz.visualize_neuron_lite(neuron_obj_1)

In [ ]:
neuron_obj_2 = neuron_list[1]
nviz.visualize_neuron_lite(neuron_obj_2)

# Iterating through the Neurons

In [ ]:
import copy
import time
neuron_obj = copy.deepcopy(neuron_obj_1)

debug_time = True
verbose = True

# Step 1: Axon Classification

In [ ]:
filter_time = time.time()

axon_limb_branch_dict,axon_angles = clu.axon_classification(neuron_obj,
                                                        return_error_labels=False,
                                                        verbose=False,
                                                        plot_axons=False,
                                                        label_axon_errors=True,
                                                        return_axon_angles=True)

if debug_time:
    print(f"Axon Classification = {time.time() - filter_time}")
    filter_time = time.time()

In [ ]:
print(f"axon_angles = {axon_angles}")

In [ ]:
import neuron_searching as ns
axon_limb_branch_dict = ns.query_neuron_by_labels(neuron_obj,
                                                 matching_labels=["axon"])
nviz.plot_limb_branch_dict(neuron_obj,
                          axon_limb_branch_dict)

In [ ]:
axon_like_limb_branch_dict = ns.query_neuron_by_labels(neuron_obj,
                                                 matching_labels=["axon-like"])
nviz.plot_limb_branch_dict(neuron_obj,
                          axon_like_limb_branch_dict)

# Classify the Neuron Cell Type

In [ ]:
clu.inhibitory_excitatory_classifier(neuron_obj,
                                     verbose=True,

axon_limb_branch_dict=axon_limb_branch_dict,
    axon_angles=axon_angles)

In [ ]:
axon_limb_branch_dict

In [ ]:
clu = reload(clu)

In [ ]:
filter_time = time.time()

(inh_exc_class,
             spine_category,
             axon_angles,
             n_axons,
             n_apicals,
             neuron_spine_density,
             n_branches_processed,
             skeletal_length_processed,
             n_branches_in_search_radius,
             skeletal_length_in_search_radius
             ) = clu.inhibitory_excitatory_classifier(neuron_obj,
                                                return_spine_classification=True,
                                                return_axon_angles=True,
                                                 return_n_axons=True,
                                                 return_n_apicals=True,
                                                 return_spine_statistics=True,
                                                     axon_limb_branch_dict_precomputed=axon_limb_branch_dict,
                                                axon_angles_precomputed=axon_angles)

if debug_time:
    print(f"Inhibitory Excitatory Classification = {time.time() - filter_time}")
    filter_time = time.time()

In [ ]:
if verbose:
    print("\n -- Cell Type Classification Results --")
    print(f"inh_exc_class={inh_exc_class}")
    print(f"spine_category={spine_category}")
    print(f"axon_angles={axon_angles}")
    print(f"n_axons={n_axons}")
    print(f"n_apicals={n_apicals}")
    print(f"neuron_spine_density={neuron_spine_density}")
    print(f"n_branches_processed={n_branches_processed}")
    print(f"skeletal_length_processed={skeletal_length_processed}")
    print(f"n_branches_in_search_radius={n_branches_in_search_radius}")
    print(f"skeletal_length_in_search_radius={skeletal_length_in_search_radius}")


# Doing the Excitatory Filtering